# Merge mapping outputs from different classifications (wikidata, dictionaries, self & spacy) -  Classifying special street designations

In [1]:
#Load all needed libraries
import pandas as pd
import time
from functools import reduce  
import warnings

Add all data

In [2]:
#Streetnames
streetnames = pd.read_csv('../Datapreparation/streetnames.csv', encoding='UTF-8-SIG', sep=';')

#Results from spacy classification
spacy = pd.read_csv('../Datapreparation/spacy_out.csv', encoding='UTF-8-SIG', sep=';')

#Results from classification using wikidata
wiki_person_de = pd.read_csv('mapping wiki person_de_out.csv', encoding='UTF-8-SIG', sep=';')
wiki_person_fr = pd.read_csv('mapping wiki person_fr_out.csv', encoding='UTF-8-SIG', sep=';')

#Results from classification using dictionaries & self-classifier
dictionary = pd.read_csv('out_dictionary.csv', encoding='UTF-8-SIG', sep=';')

In [3]:
spacy.loc[[2,12,30,32,125,54718, 54656, 54680, 54712, 54746, 54779], ['STR_ESID', 'STN_LABEL_FINAL', 'SPACY_DE_LEMMA']]

,STR_ESID,STN_LABEL_FINAL,SPACY_DE_LEMMA
2,10140563,Emanuelen,Emanuel
12,10169078,Kelten,Kelt
30,10142970,Meisen,Meisen
32,10103851,Ringlichen,Ringliche
125,10108780,Eisen,Ei
54718,10209758,Sonnhalde,Sonnhald
54656,10013619,Moselen,Mosel
54680,10116917,Häusern,Haus
54712,10064127,Alpbachsäge,Alpbachsag
54746,10119756,Truffersbrunnen,Truffersbrunn


Union de & fr

In [4]:
frames = [wiki_person_de, wiki_person_fr]
  
wikidata = pd.concat(frames)

Merge these datasets

mapping = reduce(lambda left, right:     # Merge three pandas DataFrames
                     pd.merge(left , right,
                              on = ["STR_ESID"],
                              how = "outer"),
                     [streetnames, spacy, dictionary])


# Add wiki_person

In [5]:
warnings.filterwarnings("ignore")

mapping = reduce(lambda left, right:     # Merge three pandas DataFrames
                     pd.merge(left , right,
                              on = ["STR_ESID"],
                              how = "outer"),
                     [streetnames, spacy, dictionary, wikidata])




In [6]:
mapping.loc[mapping['DIC_Ortschaft'].notna() , 'classif'] = 'Ortschaft'
mapping.loc[mapping['DIC_Berufe'].notna() , 'classif'] = 'Beruf'
mapping.loc[(mapping['SPACY_FR_ENT'] == 'PER') | (mapping['SPACY_DE_ENT'] == 'PER'), 'classif'] = 'Person'
mapping.loc[mapping['wikiQLabel'].notnull(), 'classif'] = 'Person'
mapping.loc[mapping['DIC_Lebewesen'].notna() , 'classif'] = 'Lebewesen'
mapping.loc[mapping['DIC_Siedlung'].notna() , 'classif'] = 'Siedlung'
mapping.loc[mapping['DIC_Religion'].notna() , 'classif'] = 'Religion'
mapping.loc[mapping['DIC_Natur'].notna() , 'classif'] = 'Natur'
mapping.loc[mapping['DIC_Berg'].notna() , 'classif'] = 'Berg'
mapping.loc[mapping['DIC_Gewässer'].notna() , 'classif'] = 'Gewässer'
mapping.loc[mapping['STR_PREPS'] == 'sankt', 'classif'] = 'Religion'
mapping["classif"] = mapping["classif"].fillna('Andere')

Clean output dataset

In [7]:
mapping.columns

Index(['Unnamed: 0_x', 'STR_ESID', 'STN_LABEL_x', 'COM_FOSNR', 'COM_NAME',
       'COM_CANTON', 'STR_OFFICIAL', 'STR_EASTING', 'STR_NORTHING',
       'STN_LABEL_NO_BI', 'STN_LABEL_NO_TERMS', 'STR_TERMS', 'STR_PREPS',
       'STN_LABEL_FINAL_x', 'Unnamed: 0_y', 'STN_LABEL_y', 'STN_LABEL_FINAL_y',
       'SPACY_DE', 'SPACY_DE_LEMMA', 'SPACY_DE_ENT', 'SPACY_FR',
       'SPACY_FR_LEMMA', 'SPACY_FR_ENT', 'Unnamed: 0_x', 'STN_LABEL',
       'STN_LABEL_FINAL_x', 'DIC_Lebewesen', 'DIC_Ortschaft', 'DIC_Berufe',
       'DIC_Gewässer', 'DIC_Berg', 'DIC_Siedlung', 'DIC_Religion', 'DIC_Natur',
       'Unnamed: 0_y', 'STN_LABEL_FINAL_y', 'wikiQLabel', 'wikiQ', 'sex',
       'birth', 'death', 'placebirth', 'placedeath', 'image', 'classif'],
      dtype='object')

In [8]:
mapping = mapping.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'STN_LABEL_y', 'STN_LABEL_FINAL_y','STN_LABEL_FINAL_x' ,'STN_LABEL_x', 'Unnamed: 0_y','STN_LABEL_FINAL_y' ], axis=1)

In [9]:
mapping.to_csv('C:\CAS_Arbeit\cassda-zertifikatsarbeit\Modeling\out_mapping.csv', encoding='UTF-8-SIG', sep=';')